# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [13]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [14]:
# Part 1
df_before = df.copy()

df["Measurement Timestamp"] = pd.to_datetime(df["Measurement Timestamp"], errors="coerce")

numeric_cols = [
    "Air Temperature", "Wet Bulb Temperature", "Humidity", "Rain Intensity",
    "Interval Rain", "Total Rain", "Wind Direction", "Wind Speed",
    "Maximum Wind Speed", "Barometric Pressure", "Solar Radiation",
    "Heading", "Battery Life"
]
print(df.isnull().sum())

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.drop_duplicates()
df["Rain Intensity"] = df["Rain Intensity"].fillna(0)
df["Total Rain"] = df["Total Rain"].fillna(0)
df["Precipitation Type"] = df["Precipitation Type"].fillna('None')
df["Air Temperature"]= df["Air Temperature"].interpolate()
df["Barometric Pressure"]= df["Barometric Pressure"].interpolate()
df["Wet Bulb Temperature"] = df["Wet Bulb Temperature"].fillna(
    df["Wet Bulb Temperature"].median())
df["Heading"] = df["Heading"].fillna(
    df["Heading"].median())

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    df[col] = df[col].clip(lower, upper)
    print(f"Column: {col}, Outliers handled: {len(outliers)}")


# save cleaned data
df.to_csv("output/q2_cleaned_data.csv", index=False)



Station Name                       0
Measurement Timestamp              0
Air Temperature                   75
Wet Bulb Temperature           76049
Humidity                           0
Rain Intensity                 76049
Interval Rain                      0
Total Rain                     76049
Precipitation Type             76049
Wind Direction                     0
Wind Speed                         0
Maximum Wind Speed                 0
Barometric Pressure              146
Solar Radiation                    0
Heading                        76049
Battery Life                       0
Measurement Timestamp Label        0
Measurement ID                     0
dtype: int64
Column: Air Temperature, Outliers handled: 97
Column: Wet Bulb Temperature, Outliers handled: 13305
Column: Humidity, Outliers handled: 185
Column: Rain Intensity, Outliers handled: 4250
Column: Interval Rain, Outliers handled: 15862
Column: Total Rain, Outliers handled: 26778
Column: Wind Direction, Outliers handled: 0

In [15]:
# Part 2
rows_before = len(df_before)   
missing_before = df_before.isnull().sum()

rows_after = len(df)
missing_after = df.isnull().sum()

duplicates_removed = df_before.shape[0] - df.shape[0]


missing_methods = {
    "Air Temperature": "Interpolation",
    "Wet Bulb Temperature": "Median imputation",
    "Rain Intensity": "Filled with 0",
    "Total Rain": "Filled with 0",
    "Precipitation Type": "Filled with 'None'",
    "Barometric Pressure": "Interpolation",
    "Heading": "Median imputation"
}

outlier_counts = {}
iqr_bounds = {}

for col in numeric_cols:
    Q1 = df_before[col].quantile(0.25)
    Q3 = df_before[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df_before[(df_before[col] < lower) | (df_before[col] > upper)]
    outlier_counts[col] = len(outliers)
    iqr_bounds[col] = (lower, upper)


datatype_changes = ["Measurement Timestamp → datetime64[ns]"]
for col in numeric_cols:
    datatype_changes.append(f"{col} → numeric")


with open("output/q2_cleaning_report.txt", "w") as f:
    
    f.write(f"Rows before cleaning: {rows_before}\n")
    f.write("\nMissing Data Handling:\n")

    for col, method in missing_methods.items():
        before = missing_before[col]
        pct = (before / rows_before) * 100
        f.write(f"- {col}: {before} missing values ({pct:.2f}%)\n")
        f.write(f"  Method: {method}\n")
        f.write("  Result: All missing values filled\n\n")
    
    f.write("Outlier Handling:\n")
    f.write("Detection method: IQR (1.5 × IQR rule)\n\n")

    for col in numeric_cols:
        lb, ub = iqr_bounds[col]
        f.write(f"- {col}: {outlier_counts[col]} outliers detected\n")
        f.write("  Method: Capped at IQR bounds\n")
        f.write(f"  Bounds: [{lb:.2f}, {ub:.2f}]\n")
        f.write(f"  Result: {outlier_counts[col]} values capped\n\n")

    f.write(f"Duplicates Removed: {duplicates_removed}\n\n")

    f.write("Data Type Conversions:\n")
    for line in datatype_changes:
        f.write(f"- {line}\n")
    f.write("\n")

    f.write(f"Rows after cleaning: {rows_after}\n")

print("saved output/q2_cleaning_report.txt")



saved output/q2_cleaning_report.txt


In [16]:
# Part 3

with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after))